In [2]:
#step 1: install/upgrade the latest genai SDK
%pip install google-genai --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.0/200.0 kB 7.6 MB/s eta 0:00:00


In [13]:
#import the genai library
from google import genai

In [8]:
#step 2: AIStudio: read the api key from the user data
from google.colab import userdata
client = genai.Client(api_key=userdata.get("GEMINI_KEY"))

#If you want to read from environment keys
#import os
#client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])

In [9]:
#step 4: Get model details
model_name = "models/gemini-2.5-flash-preview-05-20"
try:
    model_details = client.models.get(model=model_name) #
    print(f"Details for model '{model_name}':")
    print(f"Model Name: {model_details.name}")
    print(f"Input Token Limit: {model_details.input_token_limit}")
    print(f"Output Token Limit: {model_details.output_token_limit}")
except Exception as e:
    print(f"Error retrieving model details for '{model_name}': {e}")

Details for model 'models/gemini-2.5-flash-preview-05-20':
Model Name: models/gemini-2.5-flash-preview-05-20
Input Token Limit: 1048576
Output Token Limit: 65536


In [17]:
# Define a function declaration manually
get_weather_declaration = genai.types.FunctionDeclaration(
    name="get_weather",
    description="Get the current weather in a specific location",
    parameters={
        "type": "OBJECT",
        "properties": {
            "location": {"type": "STRING", "description": "The city and state, e.g. San Francisco, CA"}
        },
        "required": ["location"]
    }
)

# Create a Tool object containing the function declaration
weather_tool = genai.types.Tool(function_declarations=[get_weather_declaration])


In [21]:
def get_weather(location: str) -> dict:
  """Get the current weather in a specific location.

  Args:
    location: The city and state, e.g. San Francisco, CA
  Returns:
    A dictionary containing the weather information.
  """
  # (Implementation of weather fetching logic would go here)
  return {"temperature": "70F", "condition": "Sunny"}

# Infer schema from the Python function
get_weather_declaration_inferred = genai.types.FunctionDeclaration.from_callable(
    callable=get_weather,
    client=client
)

# Create a Tool object containing the inferred function declaration
weather_tool = genai.types.Tool(function_declarations=[get_weather_declaration_inferred])

In [23]:
# Configure the model to always pick a function if appropriate
config = genai.types.ToolConfig(function_calling_config={"mode": "ANY"})

# When calling the model:
# response = model.generate_content(..., tool_config=config, tools=[weather_tool])

In [24]:
specific_function_config = genai.types.ToolConfig(
    function_calling_config={
        "mode": "AUTO", # Or ANY
        "allowed_function_names": ["get_weather"]
    }
)